In [ ]:
import pandas as pd
import os
import pickle
import datetime
import telegram_send
import collections
from collections import Counter
from multiprocessing import Pool, Lock

In [ ]:
## for multicore machines, define how many cores should be used

In [ ]:
nc = int(7)

## Load the stemmed business model thesaurus and the unigrams

In [ ]:
## create a python dictionary containing the sets and descriptors for each set
## in the business model thesaurus

In [ ]:
path2bmt='/path/to/stemmed/business/model/thesaurus'

In [ ]:
df=pd.read_excel(path2bmt,index_col=None)

In [ ]:
sets=dict()
for col in df:
    sets[col]=df[col].dropna().tolist()

In [ ]:
## create a list of the filepaths of the files containing the
## text data scraped from the wayback machine

In [ ]:
path2wbm='/path/to/waybackmachine/unigrams'

In [ ]:
dflist=[os.path.join(path2wbm,df) for df in os.listdir(path2wbm) if df.startswith('df')]

## Define the function for checking which terms in the text get tagged by the descriptors 

In [ ]:
## here the function is defined that checks if the descriptors 
## appear in the text from the websites

In [ ]:
def extract_keywords(sets, wordlist, new_keys=None, new_vals=None, vals_list=None,lst2=None):
    
    if new_keys is None:
        new_keys=[]
    if new_vals is None:
        new_vals=[]
    if vals_list is None:
        vals_list=[]
    if lst2 is None:
        lst2=[]
        
    
    for key, item in sets.items():
        lst=[]
        new_keys.append(key)
        vals_lst2=[]
        vals_list=[]
        for val in item:   # loop through descriptors
            if ' ' in val: # exclude bigrams and trigrams
                continue
            vals_list.append(val)
            vals_set=set() # create a set that will include all terms from the website text that contain the descriptor
            for wrd in wordlist: # loop through the list of terms in the text from website
                
                # set 1st specific restraint
                if val == 'dashboard': 
                    if len(wrd) > 10:
                        continue
                    else:
                        if val in wrd: # what to do if descriptor is contained within term from website text
                            vals_set.add(wrd)
                            if wrd not in lst:
                                lst.append(wrd)
                        else:
                            continue
                # 2nd specific restraint
                elif val == 'plattform':
                    if wrd.startswith('aussicht') or wrd.startswith('umspann') or \
                    wrd.startswith('oel') or wrd.startswith('konverter') or \
                    wrd.startswith('montage') or wrd.startswith('bohr'):
                        continue
                    else:
                        if val in wrd: # what to do if descriptor is contained within term from website text
                            vals_set.add(wrd)
                            if wrd not in lst:
                                lst.append(wrd)
                        else:
                            continue
                # 3rd specific restraint
                elif val == 'akku':
                    if wrd.startswith('akkurat') or wrd.startswith('vakk') or wrd.startswith('akkustik'):
                        continue
                    else:
                        if val in wrd: # what to do if descriptor is contained within term from website text
                            vals_set.add(wrd)
                            if wrd not in lst:
                                lst.append(wrd)
                        else:
                            continue
                # 4th specific restraint
                elif val == 'efahrzeug' or val == 'emobilita' or val == 'wasserstoff':
                    if not wrd.startswith(val):
                        continue
                    else:
                        if val in wrd: # what to do if descriptor is contained within term from website text
                            vals_set.add(wrd)
                            if wrd not in lst:
                                lst.append(wrd)
                        else:
                            continue
                # 5th specific restraint
                elif val == 'speicher':
                    if not wrd.endswith('speicher'):
                        continue
                    elif wrd.startswith('arbeit') or wrd.startswith('daten'):
                        continue
                    else:
                        if val in wrd: # what to do if descriptor is contained within term from website text
                            vals_set.add(wrd)
                            if wrd not in lst:
                                lst.append(wrd)
                        else:
                            continue
                # 6th specific restraint
                elif key == 'Analytics' and val.startswith('analyse') or \
                key == 'Analytics' and val.startswith('daten'):
                    if not wrd.startswith(val):
                        continue
                    else:
                        if val in wrd: # what to do if descriptor is contained within term from website text
                            vals_set.add(wrd)
                            if wrd not in lst:
                                lst.append(wrd)
                        else:
                            continue
                            
                # 7th specific restraint
                elif key == 'Infrastructure Operation' and val.startswith('netz') or \
                key == 'Infrastructure Operation' and val.startswith('speicher') or \
                key == 'Infrastructure Operation' and val.startswith('leit') or \
                key == 'Infrastructure Operation' and val.startswith('schalt'):
                    if not wrd.startswith(val):
                        continue
                    else:
                        if val in wrd: # what to do if descriptor is contained within term from website text
                            vals_set.add(wrd)
                            if wrd not in lst:
                                lst.append(wrd)
                        else:
                            continue
                #8th specific restraint
                elif key == 'Sector Coupling' and val.startswith('kopplung'):
                    if not wrd.startswith(val):
                        continue
                    else:
                        if val in wrd: # what to do if descriptor is contained within term from website text
                            vals_set.add(wrd)
                            if wrd not in lst:
                                lst.append(wrd)
                        else:
                            continue
                # 9th specific restraint
                elif val == 'saas' or val == 'paas':
                    if not wrd == val:
                        continue
                    else:
                        if val in wrd: # what to do if descriptor is contained within term from website text
                            vals_set.add(wrd)
                            if wrd not in lst:
                                lst.append(wrd)
                        else:
                            continue
                # 10th specific restraint
                elif key == 'Measuring' and val.startswith('eich') or \
                key == 'Measuring' and val.startswith('erst'):
                    if not wrd.startswith(val):
                        continue
                    else:
                        if val in wrd: # what to do if descriptor is contained within term from website text
                            vals_set.add(wrd)
                            if wrd not in lst:
                                lst.append(wrd)
                        else:
                            continue
                # 11th specific restraint
                elif key == 'Engineering & Construction' and val.endswith('anlage'):
                    if not wrd.endswith('anlage') or wrd.endswith('anlagen'):
                        continue
                    else:
                        if val in wrd: # what to do if descriptor is contained within term from website text
                            vals_set.add(wrd)
                            if wrd not in lst:
                                lst.append(wrd)
                        else:
                            continue
                # 12th specific restraint
                elif key == 'Conventional Production':
                    if wrd.startswith('bio') or wrd.startswith('oeko') or wrd.startswith('natur') or wrd.startswith('blockheiz'):
                        continue
                    elif val.startswith('gas'):
                        if wrd.startswith(val):
                            if val in wrd: # what to do if descriptor is contained within term from website text
                                vals_set.add(wrd)
                                if wrd not in lst:
                                    lst.append(wrd)
                            else:
                                continue
                        else:
                            continue
                                
                    elif val == 'kohle' or val == 'sprit' or val == 'torf' or val == 'uran':
                        if not wrd == val:
                            continue
                        else:
                            if val in wrd: # what to do if descriptor is contained within term from website text
                                vals_set.add(wrd)
                                if wrd not in lst:
                                    lst.append(wrd)
                            else:
                                continue
                    elif len(val) < 4: # if length of the descriptor is less than 4 then the length of the term from website text must be equal to the length of the descriptor
                        if val == wrd: # what to do if descriptor is contained within term from website text
                            vals_set.add(wrd)
                            if wrd not in lst:
                                lst.append(wrd)
                        else:
                            continue
                                    
                    else:
                        if val in wrd: # what to do if descriptor is contained within term from website text
                            vals_set.add(wrd)
                            if wrd not in lst:
                                lst.append(wrd)
                        else:
                            continue
                
                
                # general restraint
                elif len(val) < 4: # if length of the descriptor is less than 4 then the length of the term from website text must be equal to the length of the descriptor
                    if val == wrd: # what to do if descriptor is contained within term from website text
                        vals_set.add(wrd)
                        if wrd not in lst:
                            lst.append(wrd)
                    else:
                        continue
                    
                else:
                    if val in wrd: # what to do if descriptor is contained within word from website text
                        vals_set.add(wrd)
                        if wrd not in lst:
                            lst.append(wrd)
                    else:
                        continue
            vals_lst2.append(list(vals_set))
        
        new_words2=pd.DataFrame.from_dict(dict(zip(vals_list,vals_lst2)), orient='index').T
        lst2.append(new_words2)
        
        new_vals.append(lst)
    ## create a dataframe containing all of the terms from the text data
    ## that has been tagged by descriptors
    new_words=pd.DataFrame.from_dict(dict(zip(new_keys,new_vals)), orient='index').T
    ## create a dictionary linking all of the decriptors
    ## to the terms from the text data that they tagged
    new_words_dict=dict(zip(new_keys,lst2))
    
    return new_words, new_words_dict

## Excel writer

In [ ]:
## prepare excel writer for exporting the dictionary of descriptors and
## terms tagged by them to a
## multi-spreadsheet excel file

In [ ]:
from pandas import ExcelWriter

In [ ]:
def save_xls(dict_df, path):
    writer = ExcelWriter(path)
    for key, item in dict_df.items():
        item.to_excel(writer, key,index=False)
        
    writer.save()

## Apply functions to all files

In [ ]:
outputpath='/path/to/output/folder'

In [ ]:
keywords_tot=pd.DataFrame()
keywords_dict_tot= collections.defaultdict(list)
already_processed=[]

In [ ]:
# create a temporary file to store results dataframe
keywords_tot.to_pickle('kwt_temp.p')

In [ ]:
# create a temporary file to store results dictionary
with open('kwdt_temp.p','wb') as f:
        pickle.dump(keywords_dict_tot,f)

In [ ]:
# create a list of the files that have been already processed
with open('already_processed_1gram.p','wb') as f:
        pickle.dump(already_processed,f)

In [ ]:
## here the function is defined that will be applied to each 
## file containing the text from the websites 

In [ ]:
def extract(file):
    
    st=datetime.datetime.now()
    fname=os.path.basename(file)
    print(fname,datetime.datetime.now())
    
    with open('already_processed_1gram.p','rb') as f:
        ap=pickle.load(f)
    if fname in ap:
        print(f'File {fname} already processed')
        return
    if fname == 'df10y197.p': # leave file 197 for later because it is very large and needs all the ram for itself
        print(f'Found {fname}!!!')
        return
    
    
    df=pickle.load(open(file,'rb')) # load the file with the website text into a dataframe
    words=[]
    for col in df:
        words=[]
        for cell in df[col].dropna():
            for wrd in cell.split():
                words.append(wrd.strip())
        
        # apply the function defined in section 2 to the dataframe
        keywords, keywords_dict=extract_keywords(sets,words)
        
        # update the temporary results dataframe file
        lock.acquire()
        keywords_tot=pd.read_pickle('kwt_temp.p')
        keywords_tot=keywords_tot.append(keywords)
        keywords_tot.to_pickle('kwt_temp.p')
        lock.release()

        # update the temporary results dictionary file
        lock.acquire()
        with open('kwdt_temp.p','rb') as f:
                keywords_dict_tot=pickle.load(f)

        for key, items in keywords_dict.items():
            if not key in keywords_dict_tot:
                keywords_dict_tot[key]=items
            else:
                keywords_dict_tot[key]=pd.concat([keywords_dict_tot[key],items])
                
        with open('kwdt_temp.p','wb') as f:
            pickle.dump(keywords_dict_tot,f)
        lock.release()
    
    # update the list of files that have been already processed
    lock.acquire()
    with open('already_processed_1gram.p','rb') as f:
        ap=pickle.load(f)
        ap.append(fname)
        print(f'{len(ap)} files processed so far')
    
    with open('already_processed_1gram.p','wb') as f:
        pickle.dump(ap,f)
    lock.release()
    
    et=datetime.datetime.now()
    print(f'Time taken for {fname} was {et-st}')

In [ ]:
## here the functions are defined for assigning the above function
## to individual cores

In [ ]:
def init(l):
    global lock
    lock = l

In [ ]:
def main():
    l = Lock()
    pool = Pool(initializer=init, initargs=(l,),processes=nc)
    pool.map(extract, dflist)
    pool.close()
    pool.join()

In [ ]:
starttime=datetime.datetime.now()
main()  

In [ ]:
# file 197 is very large and needs to be processed individually due to memory restrictions
ausnahme=[x for x in dflist if '197' in x]

In [ ]:
print('Working on file 197...')

In [ ]:
file=ausnahme[0]
fname=os.path.basename(file)
print(fname,datetime.datetime.now())
df=pickle.load(open(file,'rb')) # load the file with the website text into a dataframe
words=[]
for col in df:
    words=[]
    for cell in df[col].dropna():
        for wrd in cell.split():
            words.append(wrd.strip())

            
    # apply the function defined in section 2 to the dataframe
    keywords, keywords_dict=extract_keywords(sets,words)
    
    # update the temporary results dataframe file
    keywords_tot=pd.read_pickle('kwt_temp.p')  
    keywords_tot=keywords_tot.append(keywords)
    keywords_tot.to_pickle('kwt_temp.p')
    
    # update the temporary results dictionary file
    with open('kwdt_temp.p','rb') as f:
        keywords_dict_tot=pickle.load(f)
        
    for key, items in keywords_dict.items():
        if not key in keywords_dict_tot:
            keywords_dict_tot[key]=items
        else:
            keywords_dict_tot[key]=pd.concat([keywords_dict_tot[key],items])
            
    with open('kwdt_temp.p','wb') as f:
        pickle.dump(keywords_dict_tot,f)
    
    # update the list of files that have been already processed
    with open('already_processed_1gram.p','rb') as f:
        ap=pickle.load(f)
        ap.append(fname)
    
    with open('already_processed_1gram.p','wb') as f:
        pickle.dump(ap,f)
    
et=datetime.datetime.now()

In [ ]:
endtime=datetime.datetime.now()

ttt=endtime-starttime

print(f'Total time taken was {ttt}')  

## Export the results

In [ ]:
## define output filenames and save results

In [ ]:
nw=datetime.datetime.now().strftime('%d_%m_%y@%H:%M')
kwt_out=f'/home/ensys/anaconda3/Masterarbeit/Processed Data/kwt_{nw}.xlsx'
kwdt_out=f'/home/ensys/anaconda3/Masterarbeit/Processed Data/kwdt_{nw}.xlsx'

In [ ]:
keywords_tot=pd.read_pickle('kwt_temp.p')

In [ ]:
ktf=keywords_tot.apply(lambda x: pd.Series(sorted(Counter(x.dropna()).items(), key=lambda pair: pair[1], reverse=True)))

In [ ]:
ktf.to_excel(kwt_out)

In [ ]:
with open('kwdt_temp.p','rb') as f:
            keywords_dict_tot=pickle.load(f)

In [ ]:
kwd=keywords_dict_tot.copy()

In [ ]:
for ky, df in kwd.items():
    print(len(df),ky)
    df=df.apply(lambda x: pd.Series(sorted(Counter(x.dropna()).items(), key=lambda pair: pair[1], reverse=True)))
    kwd[ky]=df
    print(len(df))

In [ ]:
save_xls(kwd,kwdt_out)

In [ ]:
## send notification of successfull completion to telegram account

In [ ]:
telegram_send.send(messages=[f'Finished extracting unigrams\nFilename={nw} \nTime taken = {ttt}'])